## Deploy a Grafana Dashboard  
To track the different stocks on a live dashboard we will use **Grafana**.  <br>
We will use [Grafwiz](https://github.com/v3io/grafwiz) to define and deploy the dashboard directly from this notebook

In [7]:
# nuclio: ignore
import nuclio

In [8]:
# nuclio: start-code

In [9]:
from grafwiz import *
import os
import v3io_frames as v3f

In [10]:
def handler(context,streamview_url,readvector_url,rnn_serving_url,v3io_container,stocks_kv,stocks_tsdb,grafana_url):
    
    context.logger.info("Initializing DataSources1")
    context.logger.info(streamview_url)
    context.logger.info(readvector_url)
    
    DataSource(name='iguazio').deploy(grafana_url, use_auth=True)
    DataSource(name='stream-viewer', frames_url=streamview_url).deploy(grafana_url, use_auth=False, overwrite=False)
    DataSource(name='read-vector', frames_url=readvector_url).deploy(grafana_url, use_auth=False, overwrite=False)
    DataSource(name='rnn-serving', frames_url=rnn_serving_url).deploy(grafana_url, use_auth=False, overwrite=False)
    
    dash = Dashboard("stocks", start='now-7d', dataSource='iguazio')
    
    # Add a symbol combo box (template) with data from the stocks table
    dash.template(name="SYMBOL", label="Symbol", query="fields=symbol;table=" + os.getenv('V3IO_USERNAME') + "/stocks/stocks_kv;backend=kv;container=users")

    # Create a table and log viewer in one row
    tbl = Table('Current Stocks Value', span=12).source(table=stocks_kv,fields=['symbol','price', 'volume','last_updated'],container=v3io_container)
    dash.row([tbl])
    
    # Create 2 charts on the second row
    metrics_row = [Graph(metric).series(table=stocks_tsdb, fields=[metric], filter='symbol=="$SYMBOL"',container=v3io_container) for metric in ['price','volume']]
    dash.row(metrics_row)
    
    log = Table('Articles Log', dataSource='stream-viewer', span=12)
    dash.row([log])

    log = Table('Vector Log', dataSource='read-vector', span=12)
    dash.row([log])
    
    log = Table('Predictions', dataSource='rnn-serving', span=12)
    dash.row([log])
    
    dash.deploy(grafana_url)
    
    return "Done"

In [11]:
def init_context(context):
    context.logger.info("init context") 

In [12]:
# nuclio: end-code

In [13]:
# create a test event and invoke the function locally
init_context(context)
event = nuclio.Event(body='')
resp = handler(context,
               "", # here you need to insert the nuclio function endpoint - streamview_url
               "", # here you need to insert the nuclio function endpoint - readvector_url
               "", # here you need to insert the nuclio function endpoint - rnn_serving_url
               "users",
               os.getenv('V3IO_USERNAME')  + 'stocks/stocks_kv',
               os.getenv('V3IO_USERNAME') + '/stocks/stocks_tsdb', 
               "http://grafana")

Python> 2021-09-29 08:29:55,910 [info] init context
Python> 2021-09-29 08:29:55,911 [info] Initializing DataSources1
Python> 2021-09-29 08:29:55,912 [info] http://default-tenant.app.dev8.lab.iguazeng.com:32272
Python> 2021-09-29 08:29:55,912 [info] http://default-tenant.app.dev8.lab.iguazeng.com:32565
Datasource iguazio already exists
Datasource iguazio created successfully
Datasource stream-viewer created successfully
Datasource read-vector created successfully
Datasource rnn-serving created successfully
Dashboard stocks created successfully


In [19]:
import mlrun
import os 
fn = mlrun.code_to_function('grafana-tryout',
                      handler='handler', kind='job', image='mlrun/mlrun:0.6.5')

fn.apply(mlrun.platforms.v3io_cred())
fn.apply(mlrun.mount_v3io())
fn.spec.build.commands = ['pip install git+https://github.com/v3io/grafwiz --upgrade', 'pip install v3io_frames', 'pip install attrs==19.1.0']
fn.deploy()

> 2021-09-29 09:29:21,647 [info] Started building image: .mlrun/func-default-grafana-tryout:latest
E0929 09:30:04.637148       1 aws_credentials.go:77] while getting AWS credentials NoCredentialProviders: no valid providers in chain. Deprecated.
	For verbose messaging see aws.Config.CredentialsChainVerboseErrors
INFO[0040] Retrieving image manifest mlrun/mlrun:0.7.0-rc7 
INFO[0042] Retrieving image manifest mlrun/mlrun:0.7.0-rc7 
INFO[0045] Built cross stage deps: map[]                
INFO[0045] Retrieving image manifest mlrun/mlrun:0.7.0-rc7 
INFO[0047] Retrieving image manifest mlrun/mlrun:0.7.0-rc7 
INFO[0049] Executing 0 build triggers                   
INFO[0049] Unpacking rootfs as cmd RUN pip install git+https://github.com/v3io/grafwiz --upgrade requires it. 
INFO[0074] RUN pip install git+https://github.com/v3io/grafwiz --upgrade 
INFO[0074] Taking snapshot of full filesystem...        
INFO[0081] cmd: /bin/sh                                 
INFO[0081] args: [-c pip install 

True

In [20]:
fn.run(project = "stocks-" + os.getenv('V3IO_USERNAME'),
       params = {"streamview_url": "",# here you need to insert the nuclio function endpoint - streamview_url
                           "readvector_url" : "",# here you need to insert the nuclio function endpoint - readvector_url
                           "rnn_serving_url" : "",# here you need to insert the nuclio function endpoint - rnn_serving_url
                           "v3io_container" : "users",
                           "stocks_kv" : os.getenv('V3IO_USERNAME')  + 'stocks/stocks_kv',
                           "stocks_tsdb" : os.getenv('V3IO_USERNAME') + '/stocks/stocks_tsdb',
                           "grafana_url" : "http://grafana"})

> 2021-09-29 09:30:58,293 [info] starting run grafana-tryout-handler uid=3520bf1e09274c16928f333c1a5aa06f DB=http://mlrun-api:8080
> 2021-09-29 09:30:58,496 [info] Job is running in the background, pod: grafana-tryout-handler-sx6dq
> 2021-09-29 09:31:05,042 [info] Initializing DataSources1
> 2021-09-29 09:31:05,042 [info] http://default-tenant.app.dev8.lab.iguazeng.com:32272
> 2021-09-29 09:31:05,042 [info] http://default-tenant.app.dev8.lab.iguazeng.com:32565
Datasource iguazio already exists
Datasource iguazio created successfully
Datasource stream-viewer already exists
Datasource stream-viewer created successfully
Datasource read-vector already exists
Datasource read-vector created successfully
Datasource rnn-serving already exists
Datasource rnn-serving created successfully
Dashboard stocks created successfully
> 2021-09-29 09:31:05,268 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
stocks-dani,...1a5aa06f,0,Sep 29 09:31:05,completed,grafana-tryout-handler,v3io_user=danikind=jobowner=danihost=grafana-tryout-handler-sx6dq,,streamview_url=http://default-tenant.app.dev8.lab.iguazeng.com:32272readvector_url=http://default-tenant.app.dev8.lab.iguazeng.com:32565rnn_serving_url=http://default-tenant.app.dev8.lab.iguazeng.com:31789v3io_container=usersstocks_kv=danistocks/stocks_kvstocks_tsdb=dani/stocks/stocks_tsdbgrafana_url=http://grafana,return=Done,


> 2021-09-29 09:31:07,690 [info] run executed, status=completed
